In [43]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sns
import mne
from tqdm import tqdm
mne.set_log_level('ERROR')
from mne_icalabel import label_components
from mne.preprocessing import ICA
from autoreject import autoreject

In [58]:
# 吸毒
# load_path=r'E:\ZKP\drug persons\eeg\export\AllFinished_2'
# save_path=r'E:\ZKP\drug persons\eeg\export\EpochData'
# file_list = []
# for root, dirs, files in os.walk(load_path):
#     for file in files:
#         if file.endswith('.fif') and "Archive" not in root and "24" in file:
#             file_list.append(os.path.join(root, file))
# file_list

['E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2401-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2402-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2403-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2405-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2406-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2407-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2408-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2409-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2410-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2411-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2412-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2413-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2414-edf_raw.fif',
 'E:\\ZKP\\drug persons\\eeg\\export\\AllFinished\\2415-edf_raw.fif',
 'E:\\ZKP\\drug pers

In [94]:
load_path=r'E:\ZKP\health persons\Addiction_EEG\export1\AllFinished_2'
save_path=r'E:\ZKP\drug persons\eeg\export\EpochData'
file_list = []
for root, dirs, files in os.walk(load_path):
    for file in files:
        if file.endswith('.fif') and "Archive" not in root and "26" not in file:
            file_list.append(os.path.join(root, file))
file_list

['E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\1201-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\1202-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\1204-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\1205-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2302-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2303-2-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2304-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2306-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2307-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2308-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_2\\2309-edf_raw.fif',
 'E:\\ZKP\\health persons\\Addiction_EEG\\export1\\AllFinished_

In [95]:
def extractEpoch(raw,tmin,tmax,mark)-> mne.Epochs:
    events = mne.events_from_annotations(raw)
    # print(events)
    event_id = {'S 1': events[1]['S  1'],'S 2': events[1]['S  2']}
    epochs = mne.Epochs(raw, events[0], event_id, tmin, tmax, baseline=None,event_repeated="drop",preload=True,verbose=False,reject=None)
    return epochs[mark]


In [91]:
import numpy as np
data_set = []
info=None
for file in tqdm(file_list):
    raw = mne.io.read_raw(file, preload=True)
    raw = raw.set_montage('standard_1020')
    epochs = extractEpoch(raw, -3.5, 0, 'S 1')
    data = epochs.get_data()
    if len(data_set) == 0:
        data_set = data
    else:
        data_set = np.concatenate((data_set, data), axis=0)
    info=epochs.info
print(data_set.shape)

100%|██████████| 20/20 [00:14<00:00,  1.37it/s]

(300, 32, 1751)


In [92]:
epochs = mne.EpochsArray(data_set, info, )
epochs.save("E:\ZKP\drug persons\eeg\export\EpochData\HC-S1_BeforeStimuli-epo.fif",overwrite=True)

In [97]:
a=mne.read_epochs("E:\ZKP\drug persons\eeg\export\EpochData\MBT-S1_AfterStimuli-epo.fif").get_data()[0][0]
b=mne.read_epochs("E:\ZKP\drug persons\eeg\export\EpochData\MBT-S1_BeforeStimuli-epo.fif").get_data()[0][0]
print(a)
print(b)

[ 1.32853799e-07  1.21892958e-06  2.09737595e-06 ... -5.57089743e-06
 -6.10888583e-06 -6.21604295e-06]
[-2.69162399e-08 -4.87083071e-07 -5.28994576e-07 ... -3.36417156e-06
 -3.47245259e-06 -2.98342638e-06]


ValueError: operands could not be broadcast together with shapes (300,32,1751) (300,32,3501) 